# Bibliotecas

In [1]:
%%time
%pip install -U aeon --quiet
%pip install aeon[all_extras] --quiet
%pip install --upgrade numba --quiet
%pip install -U sktime --quiet
%pip install -U matplotlib --quiet

# Importações
import os.path
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import accuracy_score

from aeon.transformations.collection.convolution_based import Rocket
from aeon.classification.convolution_based import RocketClassifier
from aeon.datasets import load_classification, load_from_tsfile
from aeon.transformations.collection.convolution_based import Rocket as OriginalRocket
from numba import njit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
CPU times: total: 1.88 s
Wall time: 16.1 s


# Datasets escolhidos

In [2]:
CHOSEN_DATASETS = [
    'ArrowHead',
    'DistalPhalanxTW',
    'ECG5000',
    'ElectricDevices',
   'FaceAll',
   'FiftyWords',
    'GunPoint',
    'Ham',
    'InlineSkate',
    'Lightning2',
    'OSULeaf',
    'RefrigerationDevices',
    'ShapeletSim',
    'ShapesAll',
    'SyntheticControl',
    'TwoPatterns'
]

# Rocket Distribuição Normal

In [4]:
# Função para rodar o experimento
def run_experiment1(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train") # type: ignore
    x_test, y_test = load_classification(dataset_name, split="test") # type: ignore

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = Rocket() # type: ignore
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    output_dir = 'Normal'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment1(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_normal.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par

# T-Student

## Rocket implementado (T-Student)

In [4]:
#Sobrescreve o método _generate_kernels
class RocketTStudent(OriginalRocket):
    def _generate_kernels(self, n_timepoints, num_kernels, n_channels, seed=None):
        # Configuração do estado aleatório, se especificado
        if seed is not None:
            np.random.seed(seed)
            
        candidate_lengths = np.array((7, 9, 11), dtype=np.int32)
        lengths = np.random.choice(candidate_lengths, num_kernels).astype(np.int32)
        
        num_channel_indices = np.zeros(num_kernels, dtype=np.int32)
        for i in range(num_kernels):
            limit = min(n_channels, lengths[i])
            num_channel_indices[i] = 2 ** np.random.uniform(0, np.log2(limit + 1))
        
        # Inicialização dos arrays para armazenar pesos, bias, etc.
        channel_indices = np.zeros(num_channel_indices.sum(), dtype=np.int32)
        weights = np.zeros(
            np.int32(
                np.dot(lengths.astype(np.float32), num_channel_indices.astype(np.float32))
            ),
            dtype=np.float32,
        )
        biases = np.zeros(num_kernels, dtype=np.float32)
        dilations = np.zeros(num_kernels, dtype=np.int32)
        paddings = np.zeros(num_kernels, dtype=np.int32)

        a1 = 0  # índice para pesos
        a2 = 0  # índice para channel_indices

        for i in range(num_kernels):
            _length = lengths[i]
            _num_channel_indices = num_channel_indices[i]
            
            # Geração dos pesos com a distribuição t-Student
            _weights = np.random.standard_t(df=1, size=_num_channel_indices * _length).astype(np.float32)
            
            # Centralização dos pesos
            b1 = a1 + (_num_channel_indices * _length)
            a3 = 0
            for _ in range(_num_channel_indices):
                b3 = a3 + _length
                _weights[a3:b3] = _weights[a3:b3] - _weights[a3:b3].mean()
                a3 = b3
            
            weights[a1:b1] = _weights

            # Definição dos índices dos canais e bias
            channel_indices[a2:a2 + _num_channel_indices] = np.random.choice(
                np.arange(0, n_channels), _num_channel_indices, replace=False
            )
            biases[i] = np.random.uniform(-1, 1)

            # Dilação e padding para o kernel
            dilation = 2 ** np.random.uniform(
                0, np.log2((n_timepoints - 1) / (_length - 1))
            )
            dilations[i] = np.int32(dilation)

            padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
            paddings[i] = padding

            # Atualiza índices
            a1 = b1
            a2 += _num_channel_indices

        # Retorna os parâmetros dos kernels modificados
        return (
            weights,
            lengths,
            biases,
            dilations,
            paddings,
            num_channel_indices,
            channel_indices,
        )

## Experimento

In [5]:
# Função para rodar o experimento
def run_experiment2(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train")
    x_test, y_test = load_classification(dataset_name, split="test")

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = RocketTStudent()
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    output_dir = 'T-Student'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment2(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_tstudent.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par

# Uniforme

## Rocket Uniforme

In [6]:
#Sobrescreve o método _generate_kernels
class RocketUniform(OriginalRocket):
    def _generate_kernels(n_timepoints, num_kernels, n_channels, seed):
        if seed is not None:
            np.random.seed(seed)
        candidate_lengths = np.array((7, 9, 11), dtype=np.int32)
        lengths = np.random.choice(candidate_lengths, num_kernels).astype(np.int32)

        num_channel_indices = np.zeros(num_kernels, dtype=np.int32)
        for i in range(num_kernels):
            limit = min(n_channels, lengths[i])
            num_channel_indices[i] = 2 ** np.random.uniform(0, np.log2(limit + 1))

        channel_indices = np.zeros(num_channel_indices.sum(), dtype=np.int32)

        weights = np.zeros(
            np.int32(
                np.dot(lengths.astype(np.float32), num_channel_indices.astype(np.float32))
            ),
            dtype=np.float32,
        )
        biases = np.zeros(num_kernels, dtype=np.float32)
        dilations = np.zeros(num_kernels, dtype=np.int32)
        paddings = np.zeros(num_kernels, dtype=np.int32)

        a1 = 0  # for weights
        a2 = 0  # for channel_indices

        for i in range(num_kernels):
            _length = lengths[i]
            _num_channel_indices = num_channel_indices[i]

            #_weights = np.random.normal(0, 1, _num_channel_indices * _length).astype(np.float32) #utiliza a normal
            _weights = np.random.uniform(-1, 1, _num_channel_indices * _length).astype(np.float32)  # distribuição uniforme entre -1 e 1

            b1 = a1 + (_num_channel_indices * _length)
            b2 = a2 + _num_channel_indices

            a3 = 0  # for weights (per channel)
            for _ in range(_num_channel_indices):
                b3 = a3 + _length
                _weights[a3:b3] = _weights[a3:b3] - _weights[a3:b3].mean()
                a3 = b3

            weights[a1:b1] = _weights

            channel_indices[a2:b2] = np.random.choice(
                np.arange(0, n_channels), _num_channel_indices, replace=False
            )

            biases[i] = np.random.uniform(-1, 1)#uniform distribution

            dilation = 2 ** np.random.uniform(
                0, np.log2((n_timepoints - 1) / (_length - 1))
            )
            dilation = np.int32(dilation)
            dilations[i] = dilation

            padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
            paddings[i] = padding

            a1 = b1
            a2 = b2

        return (
            weights,
            lengths,
            biases,
            dilations,
            paddings,
            num_channel_indices,
            channel_indices,
        )

## Experimento

In [7]:
# Função para rodar o experimento
def run_experiment3(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train")
    x_test, y_test = load_classification(dataset_name, split="test")

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = RocketUniform()
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    output_dir = 'Uniforme'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment3(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_uniforme.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par

# Exponencial

## Rocket Exp

In [3]:
#Sobrescreve o método _generate_kernels
class RocketExponential(OriginalRocket):
    def _generate_kernels(n_timepoints, num_kernels, n_channels, seed):
        if seed is not None:
            np.random.seed(seed)
        candidate_lengths = np.array((7, 9, 11), dtype=np.int32)
        lengths = np.random.choice(candidate_lengths, num_kernels).astype(np.int32)

        num_channel_indices = np.zeros(num_kernels, dtype=np.int32)
        for i in range(num_kernels):
            limit = min(n_channels, lengths[i])
            num_channel_indices[i] = 2 ** np.random.uniform(0, np.log2(limit + 1))

        channel_indices = np.zeros(num_channel_indices.sum(), dtype=np.int32)

        weights = np.zeros(
            np.int32(
                np.dot(lengths.astype(np.float32), num_channel_indices.astype(np.float32))
            ),
            dtype=np.float32,
        )
        biases = np.zeros(num_kernels, dtype=np.float32)
        dilations = np.zeros(num_kernels, dtype=np.int32)
        paddings = np.zeros(num_kernels, dtype=np.int32)

        a1 = 0  # for weights
        a2 = 0  # for channel_indices

        for i in range(num_kernels):
            _length = lengths[i]
            _num_channel_indices = num_channel_indices[i]

            #_weights = np.random.normal(0, 1, _num_channel_indices * _length).astype(np.float32) #utiliza a normal
            _weights = np.random.exponential(scale=1, size=_num_channel_indices * _length).astype(np.float32)

            b1 = a1 + (_num_channel_indices * _length)
            b2 = a2 + _num_channel_indices

            a3 = 0  # for weights (per channel)
            for _ in range(_num_channel_indices):
                b3 = a3 + _length
                _weights[a3:b3] = _weights[a3:b3] - _weights[a3:b3].mean()
                a3 = b3

            weights[a1:b1] = _weights

            channel_indices[a2:b2] = np.random.choice(
                np.arange(0, n_channels), _num_channel_indices, replace=False
            )

            biases[i] = np.random.uniform(-1, 1)#uniform distribution

            dilation = 2 ** np.random.uniform(
                0, np.log2((n_timepoints - 1) / (_length - 1))
            )
            dilation = np.int32(dilation)
            dilations[i] = dilation

            padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
            paddings[i] = padding

            a1 = b1
            a2 = b2

        return (
            weights,
            lengths,
            biases,
            dilations,
            paddings,
            num_channel_indices,
            channel_indices,
        )

## Experimento

In [4]:
# Função para rodar o experimento
def run_experiment4(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train")
    x_test, y_test = load_classification(dataset_name, split="test")

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = RocketExponential()
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    output_dir = 'Exponencial'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment4(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_exponencial.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par

# Cauchy

## Rocket Cauchy

In [3]:
#Sobrescreve o método _generate_kernels
class RocketCauchy(OriginalRocket):
    def _generate_kernels(n_timepoints, num_kernels, n_channels, seed):
        if seed is not None:
            np.random.seed(seed)
        candidate_lengths = np.array((7, 9, 11), dtype=np.int32)
        lengths = np.random.choice(candidate_lengths, num_kernels).astype(np.int32)

        num_channel_indices = np.zeros(num_kernels, dtype=np.int32)
        for i in range(num_kernels):
            limit = min(n_channels, lengths[i])
            num_channel_indices[i] = 2 ** np.random.uniform(0, np.log2(limit + 1))

        channel_indices = np.zeros(num_channel_indices.sum(), dtype=np.int32)

        weights = np.zeros(
            np.int32(
                np.dot(lengths.astype(np.float32), num_channel_indices.astype(np.float32))
            ),
            dtype=np.float32,
        )
        biases = np.zeros(num_kernels, dtype=np.float32)
        dilations = np.zeros(num_kernels, dtype=np.int32)
        paddings = np.zeros(num_kernels, dtype=np.int32)

        a1 = 0  # for weights
        a2 = 0  # for channel_indices

        for i in range(num_kernels):
            _length = lengths[i]
            _num_channel_indices = num_channel_indices[i]

            #_weights = np.random.normal(0, 1, _num_channel_indices * _length).astype(np.float32) #utiliza a normal
            _weights = np.random.standard_cauchy(size=_num_channel_indices * _length).astype(np.float32)

            b1 = a1 + (_num_channel_indices * _length)
            b2 = a2 + _num_channel_indices

            a3 = 0  # for weights (per channel)
            for _ in range(_num_channel_indices):
                b3 = a3 + _length
                _weights[a3:b3] = _weights[a3:b3] - _weights[a3:b3].mean()
                a3 = b3

            weights[a1:b1] = _weights

            channel_indices[a2:b2] = np.random.choice(
                np.arange(0, n_channels), _num_channel_indices, replace=False
            )

            biases[i] = np.random.uniform(-1, 1)#uniform distribution

            dilation = 2 ** np.random.uniform(
                0, np.log2((n_timepoints - 1) / (_length - 1))
            )
            dilation = np.int32(dilation)
            dilations[i] = dilation

            padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
            paddings[i] = padding

            a1 = b1
            a2 = b2

        return (
            weights,
            lengths,
            biases,
            dilations,
            paddings,
            num_channel_indices,
            channel_indices,
        )

## Exp

In [4]:
# Função para rodar o experimento
def run_experiment5(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train")
    x_test, y_test = load_classification(dataset_name, split="test")

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = RocketCauchy()
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    output_dir = 'Cauchy'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment5(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_cauchy.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par

# LogNormal

## Rocket LogNormal

In [5]:
#Sobrescreve o método _generate_kernels
class RocketLogNormal(OriginalRocket):
    def _generate_kernels(n_timepoints, num_kernels, n_channels, seed):
        if seed is not None:
            np.random.seed(seed)
        candidate_lengths = np.array((7, 9, 11), dtype=np.int32)
        lengths = np.random.choice(candidate_lengths, num_kernels).astype(np.int32)

        num_channel_indices = np.zeros(num_kernels, dtype=np.int32)
        for i in range(num_kernels):
            limit = min(n_channels, lengths[i])
            num_channel_indices[i] = 2 ** np.random.uniform(0, np.log2(limit + 1))

        channel_indices = np.zeros(num_channel_indices.sum(), dtype=np.int32)

        weights = np.zeros(
            np.int32(
                np.dot(lengths.astype(np.float32), num_channel_indices.astype(np.float32))
            ),
            dtype=np.float32,
        )
        biases = np.zeros(num_kernels, dtype=np.float32)
        dilations = np.zeros(num_kernels, dtype=np.int32)
        paddings = np.zeros(num_kernels, dtype=np.int32)

        a1 = 0  # for weights
        a2 = 0  # for channel_indices

        for i in range(num_kernels):
            _length = lengths[i]
            _num_channel_indices = num_channel_indices[i]

            #_weights = np.random.normal(0, 1, _num_channel_indices * _length).astype(np.float32) #utiliza a normal
            _weights = np.random.lognormal(0, 1, _num_channel_indices * _length).astype(np.float32)

            b1 = a1 + (_num_channel_indices * _length)
            b2 = a2 + _num_channel_indices

            a3 = 0  # for weights (per channel)
            for _ in range(_num_channel_indices):
                b3 = a3 + _length
                _weights[a3:b3] = _weights[a3:b3] - _weights[a3:b3].mean()
                a3 = b3

            weights[a1:b1] = _weights

            channel_indices[a2:b2] = np.random.choice(
                np.arange(0, n_channels), _num_channel_indices, replace=False
            )

            biases[i] = np.random.uniform(-1, 1)#uniform distribution

            dilation = 2 ** np.random.uniform(
                0, np.log2((n_timepoints - 1) / (_length - 1))
            )
            dilation = np.int32(dilation)
            dilations[i] = dilation

            padding = ((_length - 1) * dilation) // 2 if np.random.randint(2) == 1 else 0
            paddings[i] = padding

            a1 = b1
            a2 = b2

        return (
            weights,
            lengths,
            biases,
            dilations,
            paddings,
            num_channel_indices,
            channel_indices,
        )

## Experimento

In [6]:
# Função para rodar o experimento
def run_experiment6(dataset_name, num_iteracoes=50):
    # Carregar e transformar os dados de teste/treinamento
    x_train, y_train = load_classification(dataset_name, split="train")
    x_test, y_test = load_classification(dataset_name, split="test")

    # Inicializar listas para armazenar os resultados de cada iteração
    accuracy_list = []

    for i in range(num_iteracoes):
        rocket = RocketLogNormal()
        rocket.fit(x_train)

        # Transformação rocket nos dados de teste e treinamento
        x_train_transform = rocket.transform(x_train)
        x_test_transform = rocket.transform(x_test)

        # Fit classificador
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        classifier.fit(x_train_transform, y_train)

        y_pred = classifier.predict(x_test_transform)
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)

        # Adicionar os resultados da iteração às listas
        accuracy_list.append(accuracy)

    # Criar DataFrame com os resultados de cada iteração
    df_resultados = pd.DataFrame({
        'iteration': np.arange(1, num_iteracoes + 1),
        'accuracy': accuracy_list,
    })

    # Calcular médias das métricas excluindo a coluna 'iteration'
    medias = df_resultados.drop(columns=['iteration']).mean()
    desvios_padrao = df_resultados.drop(columns=['iteration']).std()

    # Criar a pasta "TLogNormal"
    output_dir = 'LogNormal'
    os.makedirs(output_dir, exist_ok=True)

    # Definir caminhos completos para os arquivos CSV dentro da pasta "TESTE"
    csv_filename = os.path.join(output_dir, f'{dataset_name}_resultados.csv')
    medias_filename = os.path.join(output_dir, f'{dataset_name}_medias.csv')
    desvios_padrao_filename = os.path.join(output_dir, f'{dataset_name}_DP.csv')

    # Salvar resultados em arquivos CSV
    df_resultados.to_csv(csv_filename, index=False)
    medias.to_csv(medias_filename, index=True, header=True)
    desvios_padrao.to_csv(desvios_padrao_filename, index=True, header=True)

    return df_resultados, medias, desvios_padrao

# Lista para armazenar os resultados de todos os datasets
resultados_gerais = []

# Executar o experimento para todos os datasets da lista CHOSEN_DATASETS
for dataset in CHOSEN_DATASETS:
    print(f"Executando experimento para o dataset: {dataset}")
    resultados_experimento, medias, desvios_padrao = run_experiment6(dataset_name=dataset, num_iteracoes=50)
    print(f"Experimento para {dataset} concluído.\n")

    # Armazenar os resultados do dataset atual
    resultados_gerais.append({
        'dataset': dataset,
        'media_accuracy': medias['accuracy'],
        'desvio_padrao_accuracy': desvios_padrao['accuracy']
    })

df_final = pd.DataFrame(resultados_gerais)

print(df_final)
df_final.to_csv("resultados_lognormal.csv", index=False)

Executando experimento para o dataset: ArrowHead
Experimento para ArrowHead concluído.

Executando experimento para o dataset: DistalPhalanxTW
Experimento para DistalPhalanxTW concluído.

Executando experimento para o dataset: ECG5000
Experimento para ECG5000 concluído.

Executando experimento para o dataset: ElectricDevices
Experimento para ElectricDevices concluído.

Executando experimento para o dataset: FaceAll
Experimento para FaceAll concluído.

Executando experimento para o dataset: FiftyWords
Experimento para FiftyWords concluído.

Executando experimento para o dataset: GunPoint
Experimento para GunPoint concluído.

Executando experimento para o dataset: Ham
Experimento para Ham concluído.

Executando experimento para o dataset: InlineSkate
Experimento para InlineSkate concluído.

Executando experimento para o dataset: Lightning2
Experimento para Lightning2 concluído.

Executando experimento para o dataset: OSULeaf
Experimento para OSULeaf concluído.

Executando experimento par